## Import Libraries

In [19]:
from tensorflow.keras.models import load_model
import pandas as pd
import os
import tqdm
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

model = load_model('full_model.h5')

Testing Data is prepared for testing the data on trained model.

In [10]:
def Data(image_folder):
    image_paths = []
    images = []
    bounding_boxes = []
    class_labels = []

    data = pd.read_csv(image_folder + '/_annotations.csv')

    for index, row in data.iterrows():
        image_path = os.path.join(image_folder, row['filename'])
        image = cv.imread(image_path) # type: ignore

        x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        class_label = row['class']

        image_paths.append(image_path)
        images.append(image)
        bounding_boxes.append((x_min, y_min, x_max, y_max))
        class_labels.append(class_label)

    return image_paths, images, bounding_boxes, class_labels

In [8]:
def resize(images, bounding_boxes, target_size):
    resized_images = []
    resized_bbox = []
    
    for i in range(len(images)):
        im = images[i]
        xmin, ymin, xmax, ymax = bounding_boxes[i]

        y_ = im.shape[0]
        x_ = im.shape[1]
        x_scale = target_size[0]/x_
        y_scale = target_size[1]/y_

        resized_image = cv.resize(im, target_size) # type: ignore

        resized_xmin = int(np.round(xmin*x_scale))
        resized_ymin = int(np.round(ymin*y_scale))
        resized_xmax = int(np.round(xmax*x_scale))
        resized_ymax = int(np.round(ymax*y_scale))

        resized_box = [resized_xmin, resized_ymin, resized_xmax, resized_ymax]

        resized_images.append(resized_image)
        resized_bbox.append(resized_box)

    return resized_images, resized_bbox

In [13]:
test_image_paths, test_images, test_bbox, test_labels = Data('test')

In [14]:
resized_test_images, resized_test_bbox = resize(test_images, test_bbox, (128,128))

In [15]:
resized_test_images, resized_test_bbox = np.array(resized_test_images)/255, np.array(resized_test_bbox)

In [ ]:
test_loss, test_accuracy = model.evaluate(resized_test_images, resized_test_bbox)

print("\nLoss on Testing data:", test_loss)
print("Accuracy on esting data:", test_accuracy)

In [ ]:
predictions = model.predict(resized_test_images)

The blue box is the true bounding box and red box is the predicted bounding box

In [ ]:
test_image_1 = resized_test_images[1].copy()
test_image_bbox = resized_test_bbox[1]
predicted_image_bbox = predictions[1]

cv.rectangle(test_image_1, (test_image_bbox[0], test_image_bbox[1]), (test_image_bbox[2], test_image_bbox[3]), (0,0,255), 2)
cv.rectangle(test_image_1, (int(predicted_image_bbox[0]), int(predicted_image_bbox[1])), (int(predicted_image_bbox[2]), int(predicted_image_bbox[3])), (255,0,0), 2)

plt.imshow(test_image_1)

In [ ]:
test_image_1 = resized_test_images[20].copy()
test_image_bbox = resized_test_bbox[20]
predicted_image_bbox = predictions[20]

cv.rectangle(test_image_1, (test_image_bbox[0], test_image_bbox[1]), (test_image_bbox[2], test_image_bbox[3]), (0,0,255), 2)
cv.rectangle(test_image_1, (int(predicted_image_bbox[0]), int(predicted_image_bbox[1])), (int(predicted_image_bbox[2]), int(predicted_image_bbox[3])), (255,0,0), 2)

plt.imshow(test_image_1)

# IOU

Intersection over Union (IoU) is a metric commonly used to evaluate the accuracy of object detection algorithms. It measures the overlap between two bounding boxes and is calculated as the area of intersection divided by the area of the union of the two boxes.

In [23]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Parameters:
    - box1: Tuple (x1, y1, x2, y2) representing the coordinates of the first bounding box.
    - box2: Tuple (x1, y1, x2, y2) representing the coordinates of the second bounding box.

    Returns:
    - iou: Intersection over Union (IoU) between the two bounding boxes.
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate coordinates of the intersection rectangle
    x_intersection = max(x1, x2)
    y_intersection = max(y1, y2)
    w_intersection = min(x1 + w1, x2 + w2) - x_intersection
    h_intersection = min(y1 + h1, y2 + h2) - y_intersection

    # Check if there is no intersection
    if w_intersection <= 0 or h_intersection <= 0:
        return 0.0

    # Calculate areas of the bounding boxes and the intersection
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    area_intersection = w_intersection * h_intersection

    # Calculate IoU
    iou = area_intersection / float(area_box1 + area_box2 - area_intersection)
    return iou


In [24]:
iou_errors = []
total_iou = 0.0
num_pairs = 0

for pred_bbox in predictions:
    for true_bbox in resized_test_bbox:
        iou = calculate_iou(pred_bbox, true_bbox)
        # print("IoU:", iou)
        total_iou += iou
        num_pairs += 1

        iou_errors.append(iou)

mean_iou = total_iou / num_pairs


In [ ]:
print("Mean IoU:", mean_iou)

The prediction on custom image is calculated using below  function

In [28]:
def predict(image_path):
    im = cv.imread(image_path)
    im = cv.resize(im, (128,128))

    im = np.array(im)/255
    image = im.reshape(1, im.shape[0], im.shape[1], im.shape[2])

    print(image.shape)
    print(im.shape)
    bbox = model.predict(image)
    print(bbox)
    cv.rectangle(im, (int(bbox[0][0]), int(bbox[0][1])), (int(bbox[0][2]), int(bbox[0][3])), (0,0,255), 2)

    plt.imshow(im)